In [1]:
import sys
sys.path.append("../../")
import os
import datetime
import pandas as pd
import numpy as np
from example_pendulum import get_pendulum_data
from sindy_utils import library_size
from training import train_network
import torch

# Generate data

In [2]:
training_data_ = get_pendulum_data(100)
validation_data_ = get_pendulum_data(10)


# Function to convert data to tensors and move to GPU
def convert_to_tensors(data_dict, device):
    for key, value in data_dict.items():
        if value is not None:
            data_dict[key] = torch.tensor(value).float().to(device)
    return data_dict

# Specify the device
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Convert training and validation data to tensors and move to GPU
training_data = convert_to_tensors(training_data_, device)
validation_data = convert_to_tensors(validation_data_, device)


# Set up model and training parameters

In [3]:
params = {}

params['input_dim'] = training_data['x'].shape[-1]
params['latent_dim'] = 1
params['model_order'] = 2
params['poly_order'] = 3
params['include_sine'] = True
params['library_dim'] = library_size(2*params['latent_dim'], params['poly_order'], params['include_sine'], True)

# sequential thresholding parameters
params['sequential_thresholding'] = True
params['coefficient_threshold'] = 0.1
params['threshold_frequency'] = 500
params['coefficient_mask'] = np.ones((params['library_dim'], params['latent_dim']))
params['coefficient_initialization'] = 'constant'

# loss function weighting
params['loss_weight_decoder'] = 1.0
params['loss_weight_sindy_x'] = 5e-4
params['loss_weight_sindy_z'] = 5e-5
params['loss_weight_sindy_regularization'] = 1e-5

params['activation'] = 'sigmoid'
params['widths'] = [128,64,32]

# training parameters
params['epoch_size'] = training_data['x'].shape[0]
params['batch_size'] = 1000
params['learning_rate'] = 1e-4

params['data_path'] = os.getcwd() + '/'
params['print_progress'] = True
params['print_frequency'] = 100

# training time cutoffs
params['max_epochs'] = 5001
params['refinement_epochs'] = 1001
print("training_data",training_data["ddx"])
print(validation_data_.keys())

training_data tensor([[0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 3.4865e-12, 2.0046e-13,
         9.9233e-15],
        [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 1.2309e-12, 6.6419e-14,
         3.0867e-15],
        [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 4.1304e-13, 2.0913e-14,
         9.1232e-16],
        ...,
        [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 1.7621e-17, 5.4466e-19,
         1.4532e-20],
        [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 3.9700e-18, 1.1520e-19,
         2.8857e-21],
        [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 8.6245e-19, 2.3501e-20,
         5.5293e-22]], device='cuda:0')
dict_keys(['t', 'x', 'dx', 'ddx', 'z', 'dz'])


# Run training experiments

In [4]:
print(training_data.keys)
num_experiments = 10
df = pd.DataFrame()
for i in range(num_experiments):
    print('EXPERIMENT %d' % i)

    params['coefficient_mask'] = torch.ones((params['library_dim'], params['latent_dim'])).to("cuda")

    params['save_name'] = 'pendulum_' + datetime.datetime.now().strftime("%Y_%m_%d_%H_%M_%S_%f")


    results_dict = train_network(training_data, validation_data, params)
    df = pd.concat([df, pd.DataFrame([{**results_dict, **params}])], ignore_index=True)

df.to_pickle('experiment_results_' + datetime.datetime.now().strftime("%Y%m%d%H%M") + '.pkl')

<built-in method keys of dict object at 0x0000023292AAB2C0>
EXPERIMENT 0
val_data tensor([[1.5475e-28, 5.1999e-28, 1.5127e-27,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [2.0172e-28, 6.7516e-28, 1.9564e-27,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [2.5108e-28, 8.3730e-28, 2.4173e-27,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        ...,
        [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 1.4105e-17, 4.3278e-19,
         1.1463e-20],
        [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 3.2892e-18, 9.4901e-20,
         2.3639e-21],
        [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 7.4116e-19, 2.0115e-20,
         4.7136e-22]], device='cuda:0')
TRAINING
ddz torch.Size([1000, 2601]) torch.Size([128, 2601])
sigmoid_derivative shape: torch.Size([1000, 128])
ddz_temp shape: torch.Size([1000, 128])
dz_temp shape: torch.Size([1000, 128])
term1 shape: torch.Size([1000, 128])
term2 shape: torch.Size([1000, 128])
ddz torch.Size([1000, 128]) torch.Size([

c:\Users\zhaox\OneDrive - Delft University of Technology\桌面\Q4\reproductory\report\repo_sindy-main\torch_result\pendulum\../..\training.py:224: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'x': torch.tensor(data['x'][idxs], dtype=torch.float32).to(device),
c:\Users\zhaox\OneDrive - Delft University of Technology\桌面\Q4\reproductory\report\repo_sindy-main\torch_result\pendulum\../..\training.py:225: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'dx': torch.tensor(data['dx'][idxs], dtype=torch.float32).to(device)
c:\Users\zhaox\OneDrive - Delft University of Technology\桌面\Q4\reproductory\report\repo_sindy-main\torch_result\pendulum\../..\training.py:230: UserWarning: To copy construct from

ddz torch.Size([1000, 2601]) torch.Size([128, 2601])
sigmoid_derivative shape: torch.Size([1000, 128])
ddz_temp shape: torch.Size([1000, 128])
dz_temp shape: torch.Size([1000, 128])
term1 shape: torch.Size([1000, 128])
term2 shape: torch.Size([1000, 128])
ddz torch.Size([1000, 128]) torch.Size([64, 128])
sigmoid_derivative shape: torch.Size([1000, 64])
ddz_temp shape: torch.Size([1000, 64])
dz_temp shape: torch.Size([1000, 64])
term1 shape: torch.Size([1000, 64])
term2 shape: torch.Size([1000, 64])
ddz torch.Size([1000, 64]) torch.Size([32, 64])
sigmoid_derivative shape: torch.Size([1000, 32])
ddz_temp shape: torch.Size([1000, 32])
dz_temp shape: torch.Size([1000, 32])
term1 shape: torch.Size([1000, 32])
term2 shape: torch.Size([1000, 32])
theta torch.Size([1000, 12])
torch.Size([12, 1]) torch.Size([12, 1])
z torch.Size([1000, 1])
ddz torch.Size([1000, 1]) torch.Size([32, 1])
sigmoid_derivative shape: torch.Size([1000, 32])
ddz_temp shape: torch.Size([1000, 32])
dz_temp shape: torch.Si